In [1]:
import sys
sys.path.append('/home/kong/houchang/course/chapter4/assignment')

In [2]:
from utils.config import *

In [3]:
import numpy as np

In [4]:
from utils.word2vec_dataload import *

In [5]:
embedding_matrix = load_embedding()

In [6]:
train_X, train_Y, test_X = load_data()

In [7]:
vocab, revere_vocab = load_vocab()

In [8]:
enc_units = 1024
batch_size = 64

In [9]:
from train import *

In [10]:
embedding_dim = embedding_matrix.shape[1]


In [11]:
def config_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)

In [12]:
config_gpu()

1 Physical GPUs, 1 Logical GPUs


In [13]:
import tensorflow as tf
from layer import *
from utils.word2vec_dataload import *
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

vocab, reverse_vocab = load_vocab()
def loss_function(real, pred):
    pad_index = vocab['PAD']
    # 判断逻辑值1和0的数量
    mask = tf.math.logical_not(tf.math.equal(real, pad_index))
    #　计算交叉熵
    loss_ = loss_object(real, pred)
    # 吧mask转换为loss的形式
    mask = tf.cast(mask, dtype=loss_.dtype)
    # 去掉没有<PAD>产生的损失值
    loss_ *= mask

    return tf.reduce_mean(loss_)


def get_checkpoint():
    checkpoint_dir = 'data/checkpoints/training_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                     encoder=encoder,
                                     decoder=decoder)
    return checkpoint, checkpoint_prefix


@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        # 1. 构建encoder
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        # 2. 复制
        dec_hidden = enc_hidden
        # 3. <START> * BATCH_SIZE
        dec_input = tf.expand_dims([vocab['START']] * batch_size, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):  # 这里相当与循环每一句话的第几个次,同时训练
            # decoder(x, hidden, enc_output)
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)  # 取出下句话的输入. 升维度为64 X 1

        batch_loss = (loss / int(targ.shape[1]))

        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss


def train_model(train_X, train_Y, BATCH_SIZE):
    EPOCHS = 10
    BUFFER_SIZE = len(train_X)
    checkpoint, checkpoint_prefix = get_checkpoint()
    steps_per_epoch = len(train_X) // BATCH_SIZE
    dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

    for epoch in range(EPOCHS):
        start = time.time()

        enc_hidden = encoder.initialize_hidden_state()
        total_loss = 0

        for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
            batch_loss = train_step(inp, targ, enc_hidden)
            total_loss += batch_loss

            if batch % 5 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                             batch,
                                                             batch_loss.numpy()))
        # saving (checkpoint) the model every 2 epochs
        if (epoch + 1) % 2 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

        print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                            total_loss / steps_per_epoch))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))



In [14]:
embedding_dim = embedding_matrix.shape[1]
enc_units = 1024
batch_size = 64
encoder = Encoder(enc_units, embedding_dim, embedding_matrix, batch_size, len(vocab))
decoder = Decoder(len(vocab), embedding_dim, enc_units, batch_size)
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')



In [15]:
BATCH_SIZE = batch_size

In [16]:
EPOCHS = 10
BUFFER_SIZE = len(train_X)
checkpoint, checkpoint_prefix = get_checkpoint()
steps_per_epoch = len(train_X) // BATCH_SIZE
dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
steps_per_epoch

In [17]:
dataset.take(0)

<TakeDataset shapes: ((64, 261), (64, 34)), types: (tf.int64, tf.int64)>

In [18]:
 dataset.take(2)

<TakeDataset shapes: ((64, 261), (64, 34)), types: (tf.int64, tf.int64)>

In [20]:
for (batch, (inp, targ)) in enumerate(dataset.take(2)):
    print(inp.shape)

(64, 261)
(64, 261)


In [21]:
inp

<tf.Tensor: id=28, shape=(64, 261), dtype=int64, numpy=
array([[33021,   606,    99, ..., 33023, 33023, 33024],
       [33021,   323,    64, ..., 33023, 33023, 33024],
       [33021,  5342,    17, ..., 33023, 33023, 33024],
       ...,
       [33021,   877,    64, ..., 33023, 33023, 33024],
       [33021,    58,   117, ..., 33023, 33023, 33024],
       [33021,    13,    69, ..., 33023, 33023, 33024]])>

In [17]:
enc_hidden = encoder.initialize_hidden_state()

In [22]:
enc_hidden.shape

TensorShape([64, 1024])

In [18]:
enc_hidden = encoder.initialize_hidden_state()
total_loss = 0

In [22]:
enc_output, enc_hidden = encoder(inp, enc_hidden)

In [23]:
dec_hidden = enc_hidden

In [25]:
dec_input = tf.expand_dims([vocab['START']] * batch_size, 1)

In [26]:
predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

InternalError: Blas GEMM launch failed : a.shape=(16704, 1024), b.shape=(1024, 1024), m=16704, n=1024, k=1024 [Op:MatMul] name: decoder/dense_1/Tensordot/MatMul/

In [29]:
decoder.embedding(dec_input)

<tf.Tensor: id=9354, shape=(64, 1, 300), dtype=float32, numpy=
array([[[ 0.00134235, -0.02397815,  0.00866288, ..., -0.00036164,
          0.03799422,  0.00770227]],

       [[ 0.00134235, -0.02397815,  0.00866288, ..., -0.00036164,
          0.03799422,  0.00770227]],

       [[ 0.00134235, -0.02397815,  0.00866288, ..., -0.00036164,
          0.03799422,  0.00770227]],

       ...,

       [[ 0.00134235, -0.02397815,  0.00866288, ..., -0.00036164,
          0.03799422,  0.00770227]],

       [[ 0.00134235, -0.02397815,  0.00866288, ..., -0.00036164,
          0.03799422,  0.00770227]],

       [[ 0.00134235, -0.02397815,  0.00866288, ..., -0.00036164,
          0.03799422,  0.00770227]]], dtype=float32)>

In [31]:
loss = 0 
for t in range(1, targ.shape[1]):  # 这里相当与循环每一句话的第几个次,同时训练
    # decoder(x, hidden, enc_output)
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

    loss += loss_function(targ[:, t], predictions)

    # using teacher forcing
    dec_input = tf.expand_dims(targ[:, t], 1)  # 取出下句话的输入. 升维度为64 X 1


In [24]:
train_model(train_X, train_Y, batch_size)

CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node concat_34/_148}}]] [Op:__inference_train_step_71693]

Function call stack:
train_step


In [35]:
steps_per_epoch

1294

In [36]:
BATCH_SIZE

64

In [37]:
inp.shape


TensorShape([64, 261])